# Dataset

In [ ]:
# Google drive folder
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Train set
!unzip drive/My\ Drive/ESA/FullDataset.zip 

In [ ]:
# First test set
!unzip drive/My\ Drive/ESA/TestData2.zip 

In [ ]:
# Final test set
!unzip drive/My\ Drive/ESA/final_evaluation_set.zip 

# Library

In [ ]:
!pip install pybaselines
!pip install pot
!pip install nestle

In [ ]:
import os
import ot
import math
import h5py
import scipy
import nestle
import numpy as np
import pandas as pd
import pybaselines
import matplotlib.pyplot as plt

from tqdm import tqdm
from pybaselines import utils
from sklearn import preprocessing
from matplotlib.cbook import delete_masked_points
from sklearn.metrics import mean_squared_error


# Functions

*Your code*

In [ ]:
def to_observed_matrix(data_file,aux_file):
    # careful, orders in data files are scambled. We need to "align them with id from aux file"
    num = len(data_file.keys())
    id_order = aux_file['planet_ID'].to_numpy()
    observed_spectrum = np.zeros((num,52,4))

    for idx, x in tqdm(enumerate(id_order), total=num):
        current_planet_id = f'Planet_{x}'
        instrument_wlgrid = data_file[current_planet_id]['instrument_wlgrid'][:]
        instrument_spectrum = data_file[current_planet_id]['instrument_spectrum'][:]
        instrument_noise = data_file[current_planet_id]['instrument_noise'][:]
        instrument_wlwidth = data_file[current_planet_id]['instrument_width'][:]
        observed_spectrum[idx,:,:] = np.concatenate([instrument_wlgrid[...,np.newaxis],
                                            instrument_spectrum[...,np.newaxis],
                                            instrument_noise[...,np.newaxis],
                                            instrument_wlwidth[...,np.newaxis]],axis=-1)
    return observed_spectrum


def standardise(arr, mean, std):
    return (arr-mean)/std

def transform_back(arr, mean, std):
    return arr*std+mean

def augment_data(arr, noise, repeat=10):
    noise_profile = np.random.normal(loc=0, scale=noise, size=(repeat,arr.shape[0], arr.shape[1]))
    #noise_profile = np.random.uniform(low=0, high=noise, size=(repeat,arr.shape[0], arr.shape[1]))
    ## produce noised version of the spectra
    aug_arr = arr[np.newaxis, ...] + noise_profile
    return aug_arr, noise_profile

def visualise_spectrum(spectrum):
    import matplotlib.pyplot as plt
    fig = plt.figure(figsize=(10,6))
    plt.errorbar(x=spectrum[:,0], y= spectrum[:,1], yerr=spectrum[:,2] )
    ## usually we visualise it in log-scale
    plt.xscale('log')
    plt.show()

In [ ]:
def light_track_metric(targets, predictions, k =100):
    """
    RMSE based Metric for light track. Compare quartiles between MCMC-based methods and model output"
    targets: The reference quartiles generated from a MCMC technique (N x 3 x num_targets,)
    predictions: The quartiles predicted by  ( N x 3 x num_targets,)
    k: constant , used to adjust the magnitude of the score. Default = 10
    
    """
    targets = targets.flatten()
    predictions = predictions.flatten()
    scaled_x = targets/targets
    scaled_x_hat = predictions/targets
    score= k*(10-np.sqrt(((scaled_x - scaled_x_hat) ** 2).mean()))
    print("score is:",score)
    return score

def load_Quartile_Table(path, order= None):
    """Read quartiles information from Quartiles Table and generate a 3D matrix 
    Args:
        path (string): path to quartiles table
        order (list, optional): order of the parameters, there is a default order if order is not given Defaults to None.
    Returns:
        _type_: quartiles matrix used for calculating the light track metric (N, 3, num_targets)
    """
    import pandas as pd
    quartiles = pd.read_csv(path)
    if order is None:
        targets = ['T','log_H2O', 'log_CO2','log_CH4','log_CO','log_NH3']
    else:
        targets = order
    quartiles_matrix =  np.zeros((len(quartiles), 3, len(targets)))
    for t_idx, t in enumerate(targets):
        for q_idx, q in enumerate(['q1','q2','q3']):
            quartiles_matrix[:,q_idx, t_idx, ] = quartiles.loc[:,t + '_' + q]
    return quartiles_matrix

def get_all_q(df):
    """get the GT values from the test set"""
    all_q_label = ['q1','q2','q3']
    all_q = np.zeros((3, len(df), 6))
    for idx, label in enumerate(all_q_label):
        q_matrix = df.loc[:, df.columns.str.endswith(label)].values
        all_q[idx] = q_matrix
    return 

In [ ]:
"""
-----Main Functions----
W2 - Wessestein-2 distance (implemented by POT package https://pythonot.github.io/)
"""
def batch_calculate(trace1_matrix, trace1_weights_matrix, trace2_hdf5, id_order = None):
    """Calculate the score for regular track from a (predicted) trace matrix and a Gound Truth hdf5 file
    Args:
        trace1_matrix (_type_): prediction from the model (N X M X 6), assumed to have the same number of trace for every examples
        trace1_weights_matrix: Weight for trace1 matrix (N X M), where N is the number of examples and M is the number of traces. Each row should sum to 1.
        trace2_hdf5 (_type_): GT trace data from a hdf5 file (do not need to open)
        id_order (arr, optional): Order of the planets (N). Defaults to None.
    Returns:
        float: score for regular track
    """
    # hdf5 requires knowledge on the order of the planets (in terms of planet id)
    # If unspecified, it will assume ascending order from 1 to N. 
    trace2 = h5py.File(trace2_hdf5,'r')
    if id_order is None:
        id_order = np.arange(len(trace1_matrix))
    else:
        pass
    # check size of the distribution 
    size = trace1_weights_matrix.shape[1]
    if (size <1000) or (size > 5000):
        print("Distribution size must be within 1000 to 5000")
        return None
    else:
        pass

    all_score = 0
    # ID_order = aux_test_data['planet_ID'].to_numpy()
    for i, val in tqdm(enumerate(id_order)):
        samples1 = trace1_matrix[i]
        samples1_weight = trace1_weights_matrix[i]
        samples2 = trace2[f'Planet_{val}']['tracedata'][:]
        samples2_weight = trace2[f'Planet_{val}']['weights'][:]
        # we assumed sample1 is from the participants and sample2 is from the GT.
        ResampledUserTrace= nestle.resample_equal(samples1,samples1_weight)
        ResampledUserWeights = np.ones(len(ResampledUserTrace))/len(ResampledUserTrace)
        one_score = calculate_w2(ResampledUserTrace, samples2,w1=ResampledUserWeights,w2=samples2_weight,normalise=True)
        all_score += one_score
    overall_mean_score = all_score/len(trace1_matrix)
    print("score is:",overall_mean_score)
    return overall_mean_score


def batch_calculate_from_file(trace1_hdf5, trace2_hdf5):
    """calculate score from two .hdf5 files."""
    # read data from hdf5
    trace1 = h5py.File(trace1_hdf5,'r')
    trace2 = h5py.File(trace2_hdf5,'r')
    trace1_keys = [p for p in trace1.keys()]
    # assume trace1 is coming from participants
    check_distribution(trace1, trace1_keys)

    all_score = 0
    # ID_order = aux_test_data['planet_ID'].to_numpy()
    for val in tqdm(trace1_keys, total=len(trace1_keys)):
        samples1 = trace1[val]['tracedata'][:]
        samples1_weight = trace1[val]['weights'][:]
        samples2 = trace2[val]['tracedata'][:]
        samples2_weight = trace2[val]['weights'][:]
        # we assumed sample1 is from the participants and sample2 is from the GT.
        ResampledUserTrace= nestle.resample_equal(samples1,samples1_weight)
        ResampledUserWeights = np.ones(len(ResampledUserTrace))/len(ResampledUserTrace)
        one_score = calculate_w2(ResampledUserTrace, samples2,w1=ResampledUserWeights,w2=samples2_weight,normalise=True)
        all_score += one_score
    overall_mean_score = all_score/len(trace1_keys)
    print("score is:",overall_mean_score)
    return overall_mean_score


def calculate_w2(trace1, trace2, w1=None, w2=None, normalise = True, bounds_matrix = None):
    """Calculate the Wessestein-2 distnace metric between two multivariate distributions
    Args:
        trace1 (array):  N x D Matrix, where N is the number of points and D is the dimensionality
        trace2 (array):  N x D Matrix, where N is the number of points and D is the dimensionality
        w1 (array, optional):  1D Array of N length. Defaults to None.
        w2 (array, optional):  1D Array of N length. Defaults to None.
        normalise (bool, optional): _description_. Defaults to True.
    Returns:
        scalar: W2 distance between two empirical probability distribution
    """ 
    
    if normalise:
        if bounds_matrix is None:
            bounds_matrix = default_prior_bounds()
        trace1 = normalise_arr(trace1,bounds_matrix )
        trace2 = normalise_arr(trace2,bounds_matrix )
    else:
        pass
    
    # calculate cost matrix
    M = ot.dist(trace1, trace2)

    # assume uniform weight if weights are not given
    if w1 is None:
        a = ot.unif(len(trace1))
    else:
        assert np.isclose(np.sum(w1),1)
        a = w1
    if w2 is None:
        b = ot.unif(len(trace2))
    else:
        assert np.isclose(np.sum(w2),1)
        b = w2
    M /= M.max()
    # numItermax controls the Max number of iteration before the solver "gives up" and return the result, 
    # recommended to use at least 100000 iterations for good results
    W2 = ot.emd2(a, b, M, numItermax=100000)
    # turn into an increasing score
    score = 1000* (1-W2)
    return score

"""General Helper Functions"""

def default_prior_bounds():
    T_range = [0,7000]
    gas1_range = [-12, -1]
    gas2_range = [-12, -1]
    gas3_range = [-12, -1]
    gas4_range = [-12, -1]
    gas5_range = [-12, -1]
    bounds_matrix = np.vstack([T_range,gas1_range,gas2_range,gas3_range,gas4_range,gas5_range])
    return bounds_matrix

def normalise_arr(arr, bounds_matrix):
    norm_arr = (arr - bounds_matrix[:,0])/(bounds_matrix[:,1]- bounds_matrix[:,0])
    norm_arr = restrict_to_prior(norm_arr)
    return norm_arr

def restrict_to_prior(arr,):
    # updates 06-oct, restrict values to default prior range 
    arr[arr<0] = 0
    arr[arr>1] = 1
    return arr

def check_distribution(File, UserPlanets):
    # updates from 06-Oct, perform various checks on the file, some of these changes are noted in submit_format.py from the baseline
    for planet in UserPlanets:
        traces = File[planet]['tracedata']
        weights = File[planet]['weights']
         ## sanity check - samples count should be the same for both
        assert len(traces) == len(weights)
        ## sanity check - weights must be able to sum to one.
        assert np.isclose(np.sum(weights),1)
        ## size of distribution must be between 1000 to 5000
        assert ((len(weights) > 1000) & (len(weights)<5000))
        ## weights must not be negatively negative
        assert np.sum(weights<0) == 0 

In [ ]:
def to_light_track_format(q1_array, q2_array, q3_array, columns = None, name="LT_submission.csv"):
    """Helper function to prepare submission file for the light track, 
    we assume the test data is arranged in assending order of the planet ID.
    Args:
        q1_array: N x 6 array containing the estimates for 16% percentile
        q2_array: N x 6 array containing the estimates for 50% percentile
        q3_array: N x 6 array containing the estimates for 84% percentile
        columns: columns for the df. default to none
    Returns:
        Pandas DataFrame object
    """
    # create empty array
    LT_submission_df = pd.DataFrame(columns= columns)
    # sanity check - length should be equal
    assert len(q1_array) == len(q2_array) == len(q3_array)
    targets_label = ['T', 'log_H2O', 'log_CO2','log_CH4','log_CO','log_NH3']
    # create columns for df
    default_quartiles = ['q1','q2','q3']
    default_columns = []
    for c in targets_label:
        for q in default_quartiles:
            default_columns.append(c+q)
    
    if columns is None:
        columns = default_columns
    for i in tqdm(range(len(q1_array))):
        quartiles_dict = {}
        quartiles_dict['planet_ID'] = i
        for t_idx, t in enumerate(targets_label):
            quartiles_dict[f'{t}_q1']= q1_array[i, t_idx]
            quartiles_dict[f'{t}_q2']= q2_array[i, t_idx]
            quartiles_dict[f'{t}_q3']= q3_array[i, t_idx]
        LT_submission_df = pd.concat([LT_submission_df, pd.DataFrame.from_records([quartiles_dict])],axis=0,ignore_index = True)
    LT_submission_df.to_csv(name,index= False)
    return LT_submission_df


def to_regular_track_format(tracedata_arr, weights_arr, name="RT_submission.hdf5"):
    """convert input into regular track format.
    we assume the test data is arranged in assending order of the planet ID.
    Args:
        tracedata_arr (array): Tracedata array, usually in the form of N x M x 6, where M is the number of tracedata, here we assume tracedata is of equal size. It does not have to be but you will need to craete an alternative function if the size is different. 
        weights_arr (array): Weights array, usually in the form of N x M, here we assumed the number of weights is of equal size, it should have the same size as the tracedata
    Returns:
        None
    """
    submit_file = name
    RT_submission = h5py.File(submit_file,'w')
    for n in range(len(tracedata_arr)):
        ## sanity check - samples count should be the same for both
        assert len(tracedata_arr[n]) == len(weights_arr[n])
        ## sanity check - weights must be able to sum to one.
        assert np.isclose(np.sum(weights_arr[n]),1)

        grp = RT_submission.create_group(f"Planet_{n}")
        pl_id = grp.attrs['ID'] = n 
        tracedata = grp.create_dataset('tracedata',data=tracedata_arr[n])         
        weight_adjusted = weights_arr[n]

        weights = grp.create_dataset('weights',data=weight_adjusted)
    RT_submission.close()

*My function*

In [ ]:
#Get N spectra elements
def get_spectra_noise(spec_matrix, N):
  spectra = spec_matrix[:N,:,1]
  wl_channels = len(spec_matrix[0,:,0])
  noise = spec_matrix[:N,:,2]

  global_mean = np.mean(spectra)
  global_std = np.std(spectra)

  global_max = np.max(spectra)
  global_min = np.min(spectra)
  # return spectra and noise
  return spectra, noise

In [ ]:
#Get N rows of aux table
def get_add_info(aux_training_data, N):
  radii = aux_training_data.iloc[:N, :].copy()
  mean_radii = radii.mean()
  std_radii = radii.std()
  # Features engineering
  radii["s"] = ((radii["star_mass_kg"]*radii["star_radius_m"]/radii["star_temperature"])/radii["planet_distance"])
  radii["a"] = radii["s"]/radii["planet_surface_gravity"]
  radii["c"] = ((radii["star_mass_kg"]*radii["star_radius_m"]**2/radii["star_temperature"])/(radii["planet_distance"]))
  radii["d"] = (  (4/3*math.pi*(radii["star_radius_m"]**3))-(4/3*math.pi*(radii["planet_radius_m"]**3)))
  radii["f"] = (radii["star_mass_kg"]/radii["planet_mass_kg"])
  radii["e"] = radii["star_temperature"]*radii["planet_distance"]/radii["planet_orbital_period"]
  radii["g"] = radii["planet_surface_gravity"]/radii["planet_mass_kg"]
  radii["gg"] = (radii["star_mass_kg"]/radii["g"])
  radii["Luminosity"] = 4*scipy.constants.pi*scipy.constants.Stefan_Boltzmann*(radii["star_radius_m"]**2)*(radii["star_temperature"]**4)
  # Delete ID
  del radii["planet_ID"]
  max_radii = radii.max()
  min_radii = radii.min()
  # Black body
  radii["Temp_black"]= ((radii["Luminosity"]*1)/(scipy.constants.pi*scipy.constants.Stefan_Boltzmann*16*(radii["planet_radius_m"]**2)))**(1/4)*(radii["planet_distance"]**(-(1/2)))
  radii["Temp_black1"]= (1.0-0.0)**(1/4)*(radii["planet_distance"]**(-(1/2)))*279
  return radii

In [ ]:
# Augmentation data
def augment_data_with_noise(spectra, noise, repeat ):
    aug_spectra, aug_noise = augment_data(spectra, noise, repeat)
    return aug_spectra, aug_noise

In [ ]:
# Standardize
def standardize(arr, noise, global_mean, global_std):
    for i in tqdm(range(0,len(arr))):      
      arr[i] = (arr[i]-global_mean)/global_std
      # Normalize between -1 and 1
      noise[i] =  (noise[i]-np.mean(noise[i]))/np.std(noise[i]) 
      noise[i] = (noise[i]-np.min(noise[i]))/(np.max(noise[i])-np.min(noise[i])) 
      noise[i] = (noise[i]*2)-1
    return arr, noise

In [ ]:
#Standard Normal Variate
def snv(input_data):
    # Define a new array and populate it with the corrected data  
    data_snv = np.zeros_like(input_data)
    for i in range(data_snv.shape[0]):
    # Apply correction
      data_snv[i,:] = (input_data[i,:] - np.mean(input_data[i,:])) / np.std(input_data[i,:])
    return (data_snv)

In [ ]:
# Augmentation data
def aug_data(training_spectra, training_noise, repeat, global_mean, global_std):
  # Augmentation 
  training_spectra, training_profile_noise = augment_data_with_noise(training_spectra, training_noise, repeat)
  training_spectra = np.swapaxes(training_spectra, 0, 1)
  # Apply SNV
  training_spectra = np.asarray( [ snv(t) for t in tqdm(training_spectra)] ) 
  print(training_spectra.shape)
  training_spectra = training_spectra.reshape(-1, spectra.shape[1])
  # Standardise
  # training_noise is training_spectra but normalized between -1 and 1
  training_spectra, training_noise = standardize(training_spectra.copy(), training_spectra.copy(), global_mean, global_std)
  training_spectra = training_spectra.reshape(-1, repeat, spectra.shape[1])
  training_noise = training_noise.reshape(-1, repeat, spectra.shape[1])
  print(training_spectra.shape) 
  # return spectra and another kind of spectra
  return training_spectra, training_noise

# Load dataset

*Load data*

In [ ]:
# Set path
training_path = 'Public/TrainingData/'
test_path = './final_evaluation_set' 
training_GT_path = os.path.join(training_path, 'Ground Truth Package')

In [ ]:
# Get hdf5 file
spectral_training_data = h5py.File(os.path.join(training_path,'SpectralData.hdf5'),"r")

In [ ]:
# Get Auxillary table
aux_training_data = pd.read_csv(os.path.join(training_path,'AuxillaryTable.csv'))
soft_label_data = pd.read_csv(os.path.join(training_GT_path, 'FM_Parameter_Table.csv'))

In [ ]:
# Get spectra
spec_matrix = to_observed_matrix(spectral_training_data,aux_training_data)
print("spectral matrix shape:", spec_matrix.shape)

In [ ]:
# Get ground truth
GT_Quartiles_path = os.path.join(training_path+"Ground Truth Package/", 'QuartilesTable.csv')
all_qs_dataset = load_Quartile_Table(GT_Quartiles_path)

# Preprocessing

In [ ]:
N = 10000
threshold = 0.95 
repeat = 350 # ( 3 * repeat ) > 1000 with the new rules I have to generate more than 1000 samples

Get target

In [ ]:
all_qs = all_qs_dataset[:N, :, 0:].copy()

Target

In [ ]:
target_labels = ['planet_temp','log_H2O','log_CO2','log_CH4','log_CO','log_NH3']

targets = soft_label_data.iloc[:N][target_labels]
num_targets = targets.shape[1]
targets_mean = targets.mean()
targets_std = targets.std()

targets_max = targets.max()
targets_min = targets.min()

**Load data**

In [ ]:
np.random.seed(0)
# Get spectra
spectra, noise = get_spectra_noise(spec_matrix[:N], N)
radii = get_add_info(aux_training_data[:N], N)
# Index for split train/validation
ind = np.random.rand(len(spectra)) < threshold
# Split train/validation
training_spectra, training_radii, training_targets, training_noise  = spectra[ind].copy(),radii[ind].copy(),all_qs[ind].copy(), noise[ind].copy()
valid_spectra, valid_radii, valid_targets, valid_noise, = spectra[~ind].copy(),radii[~ind].copy(),all_qs[~ind].copy(), noise[~ind].copy()
print(training_targets.shape)

*Preprocessing*

In [ ]:
global_mean = np.mean(training_spectra)
global_std = np.std(training_spectra)
# Preprocessing traning set
training_spectra, training_noise = aug_data(training_spectra, training_noise, repeat, global_mean, global_std)
# Preprocessing validation set
valid_spectra, valid_noise = aug_data(valid_spectra, valid_noise, repeat, global_mean, global_std)

*Plot spectra*

In [ ]:
x_axes = [i for i in range(52)]
for tt in training_noise[5]:
  plt.plot(x_axes, tt)
plt.show()

*Shape*

In [ ]:
print(training_noise.shape, training_radii.shape, training_targets.shape)

# Network temperature

In [ ]:
import tensorflow as tf
tf.random.set_seed(0)
from keras import initializers
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, Reshape, Input, Concatenate, BatchNormalization, Dropout, Conv1D,Flatten,MaxPooling1D, Conv2D, MaxPooling2D

*Task Metric*

In [ ]:
from keras import backend as K 
# https://www.ariel-datachallenge.space/ML/documentation/scoring Light Track metric
def rmse(y_true, y_pred):
  return K.sqrt(K.mean(K.square((y_true-y_pred)/y_true)))

*Scale aux table*

In [ ]:
# Generate scaler
min_max_scaler = preprocessing.MinMaxScaler()
# Scale training set
x_scaled1 = min_max_scaler.fit_transform(training_radii.copy()) 
df_data = pd.DataFrame(x_scaled1)
# Scale validation set
x_scaled2 = min_max_scaler.transform(valid_radii.copy())
df_data_test = pd.DataFrame(x_scaled2)

*Target values temperature*

In [ ]:
y = training_targets[:,:,0].copy() 
y_val = valid_targets[:,:,0].copy() 

**Model temperature**

In [ ]:
# Repeat, number of M repetitions per spectrum
def temp_model(repeat):
    # Aux table
    InputLayer1 = Input(shape=(20,))
    dd = Dense(256, activation="relu")(InputLayer1)
    dd = Dense(128, activation="relu")(dd)
    dd = Dense(64, activation="relu")(dd)
    dd = Dense(32, activation="relu")(dd)
    dd = Dense(16, activation="relu")(dd)
    dd = Dense(8, activation= 'relu')(dd)
    # Spectrum
    inputs = keras.layers.Input(shape=(repeat, 52), name="input")
    x = Reshape((repeat, 52, 1))(inputs)
    x = Conv2D(8, (1,3), activation='relu')(x)
    x = Conv2D(8, (1,3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(8, (1,3), activation='relu')(x)
    x = Conv2D(8, (1,3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = keras.layers.MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Dropout(0.25)(x,training=True)
    x = Dense(128, activation= 'relu')(x)
    x = Dense(32, activation= 'relu')(x)
    x = Dense(8, activation= 'relu')(x)
    x = Reshape((repeat, -1))(x)
    feat = tf.keras.layers.RepeatVector(repeat)(dd)
    # Merge two data
    x = tf.keras.layers.Concatenate()([x, feat])
    # BATCH | REPEAT | (16th, 50th, 84th) 
    x = keras.layers.Dense(3)(x)
    # Real output
    x = Reshape((repeat, 3, 1), name="real_output")(x)
    # Average on real output
    outputs = keras.layers.AveragePooling2D(pool_size=(repeat,1), strides=(1,1))(x)
    outputs = Reshape((3, 1))(outputs)

    return keras.models.Model(inputs=[inputs, InputLayer1], outputs=outputs)

*Print model*

In [ ]:
tmod = temp_model(repeat)
tmod.summary()

**Train temperature**

In [ ]:
tmod.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    metrics=  tf.keras.metrics.MeanAbsolutePercentageError(),
    loss = rmse
    )

tmod_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="drive/My Drive/ESA/temp_check1.ckp",
    save_weights_only=True,
    save_best_only=True,
    monitor='val_mean_absolute_percentage_error',
    mode='min')

tmod.fit([training_noise, df_data ], [y], 
          validation_data=([valid_noise, df_data_test ], [y_val]),
          batch_size=100, 
          callbacks=[tmod_checkpoint_callback],
          epochs=100, 
          shuffle=False,)


*Save model*

In [ ]:
tmod.load_weights("drive/My Drive/ESA/temp_check1.ckp")
tmod.save("drive/My Drive/ESA/model_t")

*Check metric*

In [ ]:
# Check on the average value
s_ = tmod.predict([training_noise, df_data])[:,1]
print(mean_squared_error(training_targets[:,1,0], s_))
s_ = tmod.predict([valid_noise, df_data_test])[:,1]
print(mean_squared_error(valid_targets[:,1,0], s_))

# Network chemicals

In [ ]:
def residual_block(x, filters, conv_num=3, activation="relu"):
    # residual block
    s = keras.layers.Conv2D(filters, 1, padding="same")(x)
    for i in range(conv_num - 1):
        x = keras.layers.Conv2D (filters, 3, padding="same")(x)
        x = keras.layers.Activation(activation)(x)
    x = keras.layers.Conv2D (filters, 3, padding="same")(x)
    x = tf.keras.layers.SpatialDropout2D(0.25)(x)
    x = keras.layers.Add()([x, s])
    x = keras.layers.Activation(activation)(x)
    return keras.layers.MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)


def chemicals_model():
    # Spectrum
    inputs = keras.layers.Input(shape=(repeat, 52), name="input")
    x = Reshape((repeat, 52, 1))(inputs)
    x = residual_block(x, 32, 2)
    x = residual_block(x, 64, 2)
    x = residual_block(x, 128, 3)
    # Attention
    x1 = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=1)(inputs,inputs)
    x1 = Dense(768, activation="relu")(x1)
    # BATCH | Repeat | ?
    x = Reshape((repeat, -1))(x)
    x = tf.keras.layers.Add()([x1, x])
    # Aux table
    feat_input = Input(shape=(10,))
    feat = tf.keras.layers.RepeatVector(repeat)(feat_input)
    # Merge
    x = tf.keras.layers.Concatenate()([x, feat])
    x = keras.layers.Dense(3*5)(x)
    # BATCH | Repeat | (16th, 50th, 84th) | 5 chemicals
    x = Reshape((repeat, 3, 5), name="real_output")(x)
    # Average
    outputs = keras.layers.AveragePooling2D(pool_size=(repeat,1), strides=(1,1))(x)
    outputs = Reshape((3, 5))(outputs)
    return keras.models.Model(inputs=[inputs, feat_input], outputs=outputs)

model = chemicals_model()

*Model*

In [ ]:
model.summary()

**Train**

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    metrics= [tf.keras.metrics.MeanAbsolutePercentageError()],
    loss = rmse
    )

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="drive/My Drive/ESA/model_check2.ckp",
    save_weights_only=True,
    save_best_only=True,
    monitor='val_mean_absolute_percentage_error',
    mode='min')

model.fit([training_noise, df_data[df_data.columns[:10]]], 
          [training_targets[:,:,1:]],
          validation_data=([valid_noise, df_data_test[df_data_test.columns[:10]]], 
                           [valid_targets[:,:,1:]]),
          batch_size=100, 
          callbacks=[model_checkpoint_callback],
          epochs=60,
          shuffle=False)


*Save model*

In [ ]:
model.load_weights("drive/My Drive/ESA/model_check2.ckp")
model.save('drive/My Drive/ESA/model_c')

# Load and print the model

In [ ]:
tmod = tf.keras.models.load_model("drive/My Drive/ESA/model_t", custom_objects={'rmse': rmse}) 
model = tf.keras.models.load_model('drive/My Drive/ESA/model_c', custom_objects={'rmse': rmse}) 

In [ ]:
tf.keras.utils.plot_model(model, to_file="chemicals.png", 
    rankdir="LR",
    show_layer_names=False,
    show_shapes=False)

In [ ]:
tf.keras.utils.plot_model(tmod, to_file="temperature.png", 
    rankdir="LR",
    show_layer_names=False,
    show_shapes=False)

# Test validation

In [ ]:
## select the corresponding GT for the validation data, and in the correct order.
index= np.arange(len(ind))
valid_index = index[~ind]

In [ ]:
# Get real output from chemicals
model_ = tf.keras.Model(inputs=model.input, outputs=model.get_layer("real_output").output)
# Get real output from temperature
model_t = tf.keras.Model(inputs=tmod.input, outputs=tmod.get_layer("real_output").output)

# Predict chemicals
y_pred = model_.predict([valid_noise, df_data_test[df_data_test.columns[:10]]])
# Predict temperature
y_pred_temp = model_t.predict([valid_noise, df_data_test])

# Shape
print(y_pred.shape)
print(y_pred_temp.shape)

# Average chemicals
y_pred_valid = (y_pred)
y_pred_valid = np.average(y_pred_valid, axis=1)
print(y_pred_valid.shape)

# Average temperature
y_pred_temp_valid = (y_pred_temp)
y_pred_temp_valid = np.average(y_pred_temp_valid, axis=1)

# Concatenate temperature and chemicals
y_pred_valid = np.concatenate((y_pred_temp_valid, y_pred_valid), axis=2)
print(y_pred_valid.shape)

# Light evaluate

In [ ]:
### load the ground truth
valid_GT_Quartiles = all_qs[valid_index]
valid_GT_Quartiles = np.swapaxes(valid_GT_Quartiles, 1,0)[:,:,]

Quartile prediction

In [ ]:
print(y_pred.shape)
# Concatenate real output (temperature and chemicals)
valid_q_pred11 = np.concatenate((y_pred_temp, y_pred), axis=3)
valid_q_pred11 = np.reshape(valid_q_pred11, (y_pred.shape[0], repeat*3, 6))
valid_q_pred11.shape

In [ ]:
# Get quartile
valid_q_pred = np.quantile(y_pred_valid, [0.16,0.5,0.84],axis=1)
valid_q_pred.shape

In [ ]:
# calculate!
light_track_metric(valid_GT_Quartiles, valid_q_pred, k =100) #993

# Evaluate - regular track


In [ ]:
## read trace and quartiles table 
GT_trace_path = os.path.join(training_GT_path, 'Tracedata.hdf5')

In [ ]:
print(y_pred.shape)
# Concatenate real output (temperature and chemicals)
valid_q_pred1 = np.concatenate((y_pred_temp, y_pred), axis=3)
valid_q_pred1 = np.reshape(valid_q_pred1, (y_pred.shape[0], repeat*3, 6))
valid_q_pred1.shape

In [ ]:
# Generates weights
def generate_w(t):
  a = np.reshape(t, -1)
  a = np.sort(a)
  split = len(a)//3
  max_ = a[-split:]
  np.random.shuffle(a[:-split])
  a = np.reshape(a, (3,-1))
  t = np.dstack((a[0],a[2], a[1]))
  return t

In [ ]:
# Get real output
trace1_matrix = valid_q_pred1 
# Generate normal distribution
norm = np.random.normal(loc=0.0, scale=1, size=(y_pred.shape[1], y_pred.shape[2]))
# Edit the distribution
norm = generate_w(norm)
norm = np.reshape(norm, -1)
# Only positive
norm -= norm.min()
# Sum 1
sum_norm = sum(norm)
norm = norm/sum_norm
# Generate weights
trace1_weights_matrix = np.asarray([norm for i in range(trace1_matrix.shape[0])])

In [ ]:
# Calculate new batch
batch_calculate(trace1_matrix, trace1_weights_matrix, GT_trace_path, id_order = valid_index) 

# Generate Test File


In [ ]:
test_path

In [ ]:
spec_test_data = h5py.File(os.path.join(test_path,'SpectralData.hdf5'),"r")
aux_test_data = pd.read_csv(os.path.join(test_path,'AuxillaryTable.csv'))

Load dataset

In [ ]:
test_spec_matrix = to_observed_matrix(spec_test_data,aux_test_data )
print(test_spec_matrix.shape)

**Get data**


---

*real data*

In [ ]:
spectra_test, noise_test = get_spectra_noise(test_spec_matrix, N)
radii_columns = list(aux_test_data.columns)
radii_columns[0] = "planet_ID"
aux_test_data.columns = radii_columns
radii_test = get_add_info(aux_test_data, N)

*supervisioned test it can no longer be used*

In [ ]:
spectra_supertest, noise_supertest = get_spectra_noise(spec_matrix[N:21988], N)
radii_supertest = get_add_info(aux_training_data[N:21988], N)
spectra_supertest.shape

**Preprocessing**


---


*real test*

In [ ]:
spectra_test, noise_test = aug_data(spectra_test, noise_test, repeat, global_mean, global_std)

*supervisioned test it can no longer be used*

In [ ]:
spectra_supertest, noise_supertest = aug_data(spectra_supertest, noise_supertest, repeat, global_mean, global_std)

**Scale aux table**


---
*real test*


In [ ]:
radii_test_scaled = min_max_scaler.transform(radii_test.copy())
df_radii_test = pd.DataFrame(radii_test_scaled)

*supervisioned test it can no longer be used*

In [ ]:
radii_supertest_scaled = min_max_scaler.transform(radii_supertest.copy())
radii_supertest = pd.DataFrame(radii_supertest_scaled)

**Shape**

In [ ]:
print(spectra_test.shape)
print(df_radii_test.shape)

**Prediction**


---

*real test*

In [ ]:
# Model chemicals
model_ = tf.keras.Model(inputs=model.input, outputs=model.get_layer("real_output").output)
# Model temperature
model_t = tf.keras.Model(inputs=tmod.input, outputs=tmod.get_layer("real_output").output)

# Prediction chemicals
y_pred_test = model_.predict([noise_test, df_radii_test[df_radii_test.columns[:10]]])
# Prediction temperature
y_pred_temp_test = model_t.predict([noise_test, df_radii_test]) 

# Shape
print(y_pred_test.shape)
print(y_pred_temp_test.shape)

# Average chemicals
y_pred_valid_test = y_pred_test
y_pred_valid_test = np.average(y_pred_test, axis=1)
# Average temperature
y_pred_valid_temp_test = y_pred_temp_test
y_pred_valid_temp_test = np.average(y_pred_valid_temp_test, axis=1)

print(y_pred_valid_test.shape)
# Concatenate temperature and chemicals
y_pred_valid_test = np.concatenate((y_pred_valid_temp_test, y_pred_valid_test), axis=2)
print(y_pred_valid_test.shape)

# Quartiles Test file

In [ ]:
print(y_pred_test.shape)
print(y_pred_temp_test.shape)
# Concatenate temperature and chemicals
q_test = np.concatenate((y_pred_temp_test, y_pred_test), axis=3)
q_test = np.reshape(q_test, (y_pred_test.shape[0], repeat*3, 6))
q_test.shape

**LT**

In [ ]:
# Quartiles
valid_q_pred = np.quantile(q_test, [0.16,0.5,0.84],axis=1)
print(valid_q_pred.shape)
# Generate LT
LT_submission = to_light_track_format(valid_q_pred[0], valid_q_pred[1], valid_q_pred[2])

**RT**

In [ ]:
print(y_pred_test.shape)
# Concaneta real output temperature and chemicals
valid_q_pred1_test = np.concatenate((y_pred_temp_test, y_pred_test), axis=3)
valid_q_pred1_test = np.reshape(valid_q_pred1_test, (y_pred_test.shape[0], repeat*3, 6))
valid_q_pred1_test.shape

In [ ]:
# Quartiles
y_pred_org = valid_q_pred1_test
# Generates weights
norm = np.random.normal(loc=0.0, scale=1, size=(y_pred_test.shape[1], y_pred_test.shape[2]))
# Edit the distribution
norm = generate_w(norm)
norm = np.reshape(norm, -1)
# Only positive
norm -= norm.min()
# Sum 1
sum_norm = sum(norm)
norm = norm/sum_norm
# Generate weights
weight = np.asarray([norm for i in range(y_pred_org.shape[0])])
print(weight.shape)
# Generate RT
RT_submission = to_regular_track_format(y_pred_org, 
                                        weight, 
                                        name="RT_submission.hdf5")

# Control of generated files

*LT*

In [ ]:
LL = pd.read_csv("LT_submission.csv")
LL.head()

*RT*

In [ ]:
f = h5py.File("RT_submission.hdf5",'r')
f['Planet_0']['tracedata'][:]